<a href="https://colab.research.google.com/github/Soujanyad10/twitter-sentiment-analysis/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import re
import tweepy
import os
from tweepy import OAuthHandler
from textblob import TextBlob
import time
from getpass import getpass
from google.colab import userdata

class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # attempt authentication
        try:
            # putting bearer token here
            BEARER_TOKEN = userdata.get('TWITTER_BEARER_TOKEN')
            self.client = tweepy.Client(bearer_token=BEARER_TOKEN)
        except Exception as e:
            print(f"Error: Authentication Failed - {e}")
            self.client = None # Initialize client to None if authentication fails

    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

    def get_tweets(self, query, count = 10):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
        next_token = None

        if self.client is None: # Check if client was initialized
            print("Twitter client not initialized. Cannot fetch tweets.")
            return None # Return None or an empty list as appropriate

        while(len(tweets) < count):
          remaining = count - len(tweets)
          try:
            response = self.client.search_recent_tweets(
                query=query,
                max_results=min(50, remaining),
                tweet_fields=["public_metrics", "lang"],
                next_token=next_token
            )
          except tweepy.errors.TooManyRequests as e:
            if "Usage cap exceeded" in str(e):
              print("❌ Monthly usage cap exceeded. Cannot fetch more tweets until quota resets or you upgrade.")
              break

            else:
              print("⚠️ Rate limit hit. Waiting for 30 days...")
              time.sleep(60) # Wait for 60 seconds before retrying
              continue # Continue to the next iteration after waiting

          except Exception as e: # Catch other potential errors during tweet fetching
            print(f"Error fetching tweets: {e}")
            break # Exit the loop on other errors


          if not response.data:
            break

          tweets.extend(response.data)

          next_token = response.meta.get("next_token")
          if not next_token:
            break

          time.sleep(5) # Add a delay between requests within the loop

        # Now, parse the tweets collected in the while loop
        parsed_tweets = []
        for tweet in tweets:
            parsed_tweet = {}
            parsed_tweet['text'] = tweet.text
            parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
            parsed_tweets.append(parsed_tweet)

        return parsed_tweets


def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    # calling function to get tweets
    tweets = api.get_tweets(query = 'Pathan', count = 200)

    # picking positive tweets from tweets
    if tweets is None:
      print("The get_tweets function returned None.")
    elif not tweets: # Handle the case where the list of tweets is empty
      print("No tweets found for the given query.")
    else:
      ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
      # percentage of positive tweets
      print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
      # picking negative tweets from tweets
      ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
      # percentage of negative tweets
      print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
      # percentage of neutral tweets
      print("Neutral tweets percentage: {} % \
          ".format(100*(len(tweets) -(len( ntweets )+len( ptweets)))/len(tweets)))

      # printing first 5 positive tweets
      print("\n\nPositive tweets:")
      for tweet in ptweets[:10]:
          print(tweet['text'])

      # printing first 5 negative tweets
      print("\n\nNegative tweets:")
      for tweet in ntweets[:10]:
          print(tweet['text'])

if __name__ == "__main__":
    # calling main function
    main()

Rate limit exceeded. Waiting for 60 seconds. Error: 429 Too Many Requests
Too Many Requests


KeyboardInterrupt: 